# New Restaurant Location Recommendations - The Johannesburg Edition

## 1. Introduction and Business Problem

The City of Johannesburg is the largest metropolitan in South Africa and is the economic capital of the country. It has a population of 4,434,827 as per the official 2011 Census. The municipality covers an area of 1,645 square kilometres stretching from Orange Farm in the south to Midrand in the north, and is made up of several urban centres that include Johannesburg, Midrand, Roodepoort, Diepsloot, Killarney, Melrose Arch, Randburg, Rosebank, Sandton, Soweto, and Sunninghill.
As the economic capital of the country, and by extension the Gauteng Province, the city has a lot of inbound migration from individuals and entrepreneurs seeking work, business and economic opportunities in general. In addition to migrants from other cities and provinces within South Africa, the city recieves additional migrants coming from other African countries such as Zimbabwe, Botswana, Mozambique, Lesotho, Swaziland, Nigeria and many other countries beyond the African continent. While some business intelligence services exist to inform any interested business people on possible locations of, say, new restaurants within the city, this comes at a cost. There is no inexpensive way I am aware of that would allow someone who is looking at opening a restaurant to evaluate the locations of existing restaurants within the city and locate any gaps and likely recommendations of potential areas before considering expensive and localised market research. Such evaluation of restaurants must also include the type of restaurant and the cuisine served to be able answer critical questions of knowing up-front who is the likely competition and how well are they rated by the patrons on popular platforms.


## 2. Available Data

The datasets that would be used include the following;

a. __FOURSQUARE__ Location Data <br>
The following description is mostly adapted from the Applied Data Science Capstone Course which offers a useful summary of the data and technology.<br>

Foursquare is a technology company that built a massive dataset of location data based on crowd-sourcing from people who use their app to build their dataset and add venues and complete any missing information they have in their dataset. Its location data is one the most comprehensive and accurate to appeal to service providers such as Apple Maps, Uber, Snapchat, Twitter, Garmin and many others, including over 100,000 developers, who use their platform. Foursquare can be used via a mobile device or online website. Both allow you to search for a venue of interest at a specific location of interest. The results would return all such venues, including their rating. Clicking on any of the venues allows one to explore the venue by redirecting you to its page on Foursquare that include its name, full address, working hours, menu, tips and images that users have posted about the shop and any of its drinks among other attributes. If one is interested in finding the most popular places (trending places), what is required is the area of interest only and all top venues are listed in order of rankings.
All these search activities are possible because of the comprehensive underlying database, which is available for free and available through an API. This availability, with some restrictions, anables anyone to access it via any application, which will be utilised in this project. <br>

b. __PLACES__ Database for the City of Johannesburg <br>
This data is based on the 2011 Census results which divided the city into 40 main places, and over 800 sub places. It has been scraped from the https://census2011.adrianfrith.com/ website, which provides online access to a selection of results from South Africa's Census 2011 down to the “sub place” layer of detail, as released in the Community Profile Database DVD set. As an example, the following is the head of the table of the dataset.

|Main Place|Sub Place|Population|Area (km²)|
|------|------|------|------|
|Alexandra|Alexandra Ext 1|5,267| 0.22|
|Alexandra|Alexandra Ext 10|1,865| 0.06|
|Alexandra|Alexandra Ext 11|682| 0.02|
|Alexandra|Alexandra Ext 12|353| 0.01|
|Alexandra|Alexandra Ext 13|1,164| 0.07|

The data will be the basis against which the availability of restaurants in a prticular are will be evaluating against. Initial part of the evaluation will consider if there is merit in using the sub place parameter in the various datasets, particularly for areas with extensions. <br>

c. __GEOCODER__ Package Data <br>
Geocoding is the process of converting addresses (like "1600 Amphitheatre Parkway, Mountain View, CA") into geographic coordinates (like latitude 37.423021 and longitude -122.083739), which can be use to place markers or a position a map. The Geocoder package, or any similar alternative, will be utilised to obtain the co-ordinates of teh main/sub places against which the data from Foursquare will be evaluated.

## 3. Code